<img src="./images/DLI_Header.png" width=400/>

# Fundamentals of Accelerated Data Science # 

## 03 - cuGraph Single Source Shortest Path ##

**Table of Contents**
<br>
This notebook GPU-accelerated graph analytics with cuGraph to identify the shortest path from node on the road network to every other node, both by distance, which we will demo, and by time, which you will implement. You will also visualize the results of your findings. This notebook covers the below sections:
1. [Environment](#Environment)
2. [Loading Data](#Loading-Data)
3. [Construct Graph with cuGraph](#Construct-Graph-with-cuGraph)
4. [Analyzing the Graph](#Analyzing-the-Graph)
5. [Single Source Shortest Path](#Single-Source-Shortest-Path)
6. [Analyze a Graph with Time Weights](#Analyze-a-Graph-with-Time-Weights)
    * [Exercise #1 - Step 1: Construct the Graph](#Exercise-#1---Step-1:-Construct-the-Graph)
    * [Exercise #2 - Step 2: Get Travel Times From a Node to All Others](#Exercise-#2---Step-2:-Get-Travel-Times-From-a-Node-to-All-Others)
    * [Visualize the Node Travel Times](#Visualize-the-Node-Travel-Times)

## Environment ##

In [1]:
import warnings
warnings.filterwarnings('ignore')

import cudf
import cugraph as cg

import cuxfilter as cxf
from bokeh.palettes import Magma, Turbo256, Plasma256, Viridis256

## Loading Data ##

We start by loading the road graph data you prepared for constructing a graph with cuGraph, with the long unique `nodeid` replaced with simple (and memory-efficient) integers we call the `graph_id`.

In [2]:
road_graph = cudf.read_csv('./data/road_graph.csv', dtype=['int32', 'int32', 'float32'])
road_graph.head()

,src,dst,length
0,0,129165,44.0
1,1,1678323,70.0
2,1,2372610,18.0
3,1,2483057,40.0
4,2,2,55.0


Next we load the graph-ready data you prepared that uses amount of time traveled as edge weight.

In [3]:
speed_graph = cudf.read_csv('./data/road_graph_speed.csv', dtype=['int32', 'int32', 'float32'])
speed_graph.head()

,src,dst,length_s
0,0,129165,3.280848
1,1,1678323,5.219531
2,1,2372610,1.342165
3,1,2483057,2.982589
4,2,2,4.101060


Finally we import the full `road_nodes` data set, which we will use below for visualizations.

In [4]:
road_nodes = cudf.read_csv('./data/road_nodes.csv', dtype=['str', 'float32', 'float32', 'str'])
road_nodes = road_nodes.drop_duplicates() # again, some road nodes appeared on multiple map tiles in the original source
road_nodes.head()

,node_id,east,north,type
0,id02FE73D4-E88D-4119-8DC2-6E80DE6F6594,320608.09375,870994.0000,junction
1,id634D65C1-C38B-4868-9080-2E1E47F0935C,320628.50000,871103.8125,road end
2,idDC14D4D1-774E-487D-8EDE-60B129E5482C,320635.46875,870983.8750,junction
3,id51555819-1A39-4B41-B0C9-C6D2086D9921,320648.68750,871083.5625,junction
4,id9E362428-79D7-4EE3-B015-0CE3F6A78A69,320658.18750,871162.3750,junction


In [5]:
road_nodes.shape

(3078117, 4)

In [6]:
speed_graph.src.max()

3078116

## Construct Graph with cuGraph ##

Now that we have the well-prepped `road_graph` data, we pass it to cuGraph to create our graph data structure, which we can then use for accelerated analysis. In order to do so, we first use cuGraph to instantiate a `Graph` instance, and then pass the instance edge sources, edge destinations, and edge weights, currently the length of the roads.

In [7]:
G = cg.Graph()
%time G.from_cudf_edgelist(road_graph, source='src', destination='dst', edge_attr='length')

CPU times: user 170 ms, sys: 28.8 ms, total: 199 ms
Wall time: 198 ms


## Analyzing the Graph ##

First, we check the number of nodes and edges in our graph:

In [8]:
G.number_of_nodes()

3078117

In [9]:
G.number_of_edges()

3620793

We can also analyze the degrees of our graph nodes. We would expect, as before, that every node would have a degree of 2 or higher, since undirected edges count as two edges (one in, one out) for each of their nodes.

In [10]:
deg_df = G.degree()
deg_df['degree'].describe()[1:]

mean     4.689990
std      1.913452
min      2.000000
25%      2.000000
50%      6.000000
75%      6.000000
max     16.000000
Name: degree, dtype: float64

We would also expect that every degree would be a multiple of 2, for the same reason. We check that there are no nodes with odd degrees (that is, degrees with a value of 1 modulo 2):

In [11]:
deg_df[deg_df['degree'].mod(2) == 1]

,degree,vertex


Observe for reference that some roads loop from a node back to itself:

In [12]:
road_graph.loc[road_graph.src == road_graph.dst]

,src,dst,length
4,2,2,55.0
145,62,62,108.0
293,124,124,67.0
471,196,196,26.0
571,240,240,44.0
...,...,...,...
7216602,3077469,3077469,78.0
7216735,3077519,3077519,111.0
7216849,3077567,3077567,69.0
7217091,3077670,3077670,30.0


## Single Source Shortest Path ##

To demo the Single Source Shortest Path (SSSP) algorithm, we will start with the node with the highest degree. First we obtain its `graph_id`, reported by the `degree` method as `vertex`:

In [13]:
demo_node = deg_df.nlargest(1, 'degree')
demo_node_graph_id = demo_node['vertex'].iloc[0]
demo_node_graph_id

652907

We can now call `cg.sssp`, passing it the entire graph `G`, and the `graph_id` for our selected vertex. Doing so will calculate the shortest path, using the road length weights we have provided, to *every* other node in the graph - millions of paths, in seconds:

In [14]:
%time shortest_distances_from_demo_node = cg.sssp(G, demo_node_graph_id)
shortest_distances_from_demo_node.head()

CPU times: user 7.59 s, sys: 25.1 ms, total: 7.61 s
Wall time: 7.59 s


,distance,vertex,predecessor
0,0.0,652907,-1
1,110322.0,1252792,1378560
2,213691.0,1606375,1375709
3,106434.0,1826781,2377706
4,253196.0,1990110,1652530


In [15]:
# Limiting to those nodes that were connected (within ~4.3 billion meters because
# cg.sssp uses the max int value for unreachable nodes, such as those on different islands)
shortest_distances_from_demo_node['distance'].loc[shortest_distances_from_demo_node['distance'] < 2**32].describe()[1:]

mean    210086.107365
std     137145.769283
min          0.000000
25%     125054.500000
50%     181815.500000
75%     252472.250000
max     868870.500000
Name: distance, dtype: float64

## Analyze a Graph with Time Weights ##

For this exercise, you are going to analyze the graph of GB's roads, but this time, instead of using raw distance for a road's weights, you will be using how long it will take to travel along the road.

### Exercise #1 - Step 1: Construct the Graph ###

Construct a cuGraph graph called `G_ex` using the sources and destinations found in `speed_graph`, along with length in seconds values for the edges' weights.

In [16]:

G_ex = cg.Graph()
G_ex.from_cudf_edgelist(speed_graph, source='src', destination='dst', edge_attr='length_s')

Click ... for solution. 

### Exercise #2 - Step 2: Get Travel Times From a Node to All Others ###

Choose one of the nodes and calculate the time it would take to travel from it to all other nodes via SSSP, calling the results `ex_dist`.

In [17]:

# If you have time, see what the SSSP visualization looks like starting from nodes at different extreme coordinates,
# or one of the end nodes of an especially long edge, or even one of the nodes unreachable from the main road network.
ex_deg = G_ex.degree()
ex_node = ex_deg.nlargest(1, 'degree')

%time ex_dist = cg.sssp(G_ex, ex_node['vertex'].iloc[0])

# limiting to those nodes that were connected (within ~4.3 billion seconds; .sssp uses the max int value for unconnected nodes)
ex_dist['distance'].loc[ex_dist['distance'] < 2**32].describe()[1:]

CPU times: user 5.88 s, sys: 28.1 ms, total: 5.91 s
Wall time: 5.87 s


mean     7424.900285
std      4666.250239
min         0.000000
25%      4484.137695
50%      6451.895508
75%      9064.045410
max     31424.103516
Name: distance, dtype: float64

Click ... for solution. 

### Visualize the Node Travel Times ###

In order to create a graphic showing the road network by travel time from the selected node, we first need to align the just-calculated distances with their original nodes. For that, we use the mapping from `node_id` strings to their `graph_id` integers.

In [18]:
mapping = cudf.read_csv('./data/node_graph_map.csv')
mapping.head()

,node_id,graph_id
0,id000000F5-5180-4C03-B05D-B01352C54F89,0
1,id000003F8-9E09-4829-AD87-6DA4438D22D8,1
2,id000010DA-C89A-4198-847A-6E62815E038A,2
3,id000017A0-1843-4BC7-BCF7-C943B6780839,3
4,id00001B2A-155F-4CD3-8E06-7677ADC6AF74,4


We see that the `sssp` algorithm has put the `graph_id`s in the `vertex` column, so we will merge on that.

In [19]:
ex_dist.head()

,distance,vertex,predecessor
0,0.000000,652907,-1
1,3753.216553,1252792,423201
2,7388.645020,1606375,1311665
3,4061.988281,1826781,1963186
4,9050.323242,1990110,1652530


In [20]:
road_nodes = road_nodes.merge(mapping, on='node_id')
road_nodes = road_nodes.merge(ex_dist, left_on='graph_id', right_on='vertex')
road_nodes.head()

,node_id,east,north,type,graph_id,distance,vertex,predecessor
0,id04231832-AEB2-4C5E-BD56-CAFFAE5D1BA7,371832.03125,862046.5000,junction,49786,23946.572266,49786,2496013
1,id2483B2D5-98C8-4458-8EE9-610FBB6434E1,371598.93750,862171.0625,road end,438690,23966.257812,438690,49786
2,idAAA610C1-858C-4629-A97B-7700DD546C04,373089.59375,862270.2500,junction,2051698,24055.251953,2051698,154149
3,idEDE3D1D7-86EE-477F-8A97-22949D207527,372495.09375,862283.5000,junction,2859739,24007.380859,2859739,2355879
4,idDE57AF33-B7EA-4CB7-A741-AD58F1CCC267,371846.00000,862292.0000,junction,2672778,23955.781250,2672778,49786


Next, we select those columns we are going to use for the visualization.

For color-scaling purposes, we get rid of the unreachable nodes with their extreme distances, and we invert the distance numbers so that brighter pixels indicate closer locations.

In [21]:
gdf = road_nodes[['east', 'north', 'distance']]
gdf = gdf[gdf['distance'] < 2**32]
gdf['distance'] = gdf['distance'].pow(1/2).mul(-1)

Otherwise, this visualization will be largely similar to the scatter plots we made to visualize the population, but instead of coloring by point density as in those cases, we will color by mean travel time to the nodes within a pixel.

In [22]:
cxf_data = cxf.DataFrame.from_dataframe(gdf)

In [23]:
heatmap_chart = cxf.charts.datashader.scatter(x='east', y='north', 
                                              # color_palette=Plasma256, # try also Turbo256, Viridis256, Magma, Plasma256
                                              # pixel_shade_type='linear', # can also be log, cbrt, linear
                                              aggregate_col='distance',
                                              aggregate_fn='mean',
                                              # point_shape='square',
                                              point_size=1)

In [24]:
dash = cxf_data.dashboard([heatmap_chart], theme=cxf.themes.dark, data_size_widget=True)

dash.app()

GridSpec(ncols=12, nrows=1)
    [0] GridSpec(height=800, ncols=12, nrows=5, sizing_mode='fixed', width=1200)
        [0] HoloViews(DynamicMap, height=800, sizing_mode='stretch_both', width=1200)
    [1] WidgetBox(styles={'border-color': '...})
        [0] Number(css_classes=['indicator'], default_color='#2B2B2B', font_size='18pt', format='{value:,}', name='Datapoints Selected', sizing_mode='stretch_width', title_size='14pt', value=3048671)
        [1] Progress(sizing_mode='stretch_width', styles={'--success-bg-color': '...}, value=100)

In [25]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

**Well Done!** Let's move to the [next notebook](2-04_networkx_cugraph.ipynb). 

<img src="./images/DLI_Header.png" width=400/>